# Bridge L3.M8.2 → L3.M8.3 Readiness Validation

**From EXPERIMENTATION TO AUTOMATION**

## Purpose

M8.2 gave you **manual A/B testing**: traffic splitting, multi-metric tracking, statistical significance, and gradual rollouts. But production systems face 5-10 commits daily from multiple engineers—**manual testing cannot scale.**

M8.3 introduces **automated CI/CD pipelines** that catch regressions before they reach production. This bridge validates you have the foundational infrastructure (golden test sets, Git workflows, containerization, fast tests) needed to automate regression detection.

## Concepts Covered

- **Readiness checks** for golden test sets, version control, containerization, and fast test execution
- **Offline-friendly validation** with graceful skips when infrastructure is missing
- **Call-forward preview** of M8.3 automation capabilities (GitHub Actions, regression baselines, DVC versioning, cost/latency guards)

## After Completing

You will be able to:

- ✅ Assess which automation prerequisites you already have (golden sets, Git, Docker, fast tests)
- ✅ Understand why manual testing breaks at scale (compounding failure scenario)
- ✅ Articulate what M8.3 will automate (GitHub Actions, regression detection, model versioning, cost guards)
- ✅ Identify gaps to address before implementing CI/CD pipelines

## Context in Track

**Bridge**: L3.M8.2 → L3.M8.3  
**Previous**: M8.2 — Experimentation (A/B testing, multi-metric evaluation, gradual rollouts)  
**Next**: M8.3 — Automation (GitHub Actions, regression detection, DVC, cost/latency guards)

## Run Locally

**Windows** (PowerShell):  
```powershell
$env:PYTHONPATH="$PWD"; jupyter notebook Bridge_L3_M8_2_to_M8_3_Readiness.ipynb
```

**macOS/Linux**:  
```bash
export PYTHONPATH=$PWD && jupyter notebook Bridge_L3_M8_2_to_M8_3_Readiness.ipynb
```

**All platforms**: Run **Cell → Run All** to execute all checks.

---
## Section 1: M8.2 Recap — What You Mastered

The previous module (M8.2) equipped you with:

### 🎯 Traffic Splitting Infrastructure
Hash-based user assignment ensures consistent routing:
- User A always gets control variant
- User B always gets treatment variant
- No cross-contamination between experiment groups

### 📊 Multi-Dimensional Measurement
Tracking 7+ simultaneous metrics:
- **Quality**: Faithfulness, Answer Relevance, Context Precision/Recall
- **Performance**: P95 latency, error rate
- **Economics**: Cost per query
- **User Satisfaction**: User ratings/feedback

### 📈 Statistical Significance Testing
Run experiments (e.g., chunk size 512 vs. 1024 tokens) with:
- P-value calculation to determine significance
- Confidence intervals for metric deltas
- Sample size planning

### 🚀 Gradual Rollout Strategy
Canary deployments with instant rollback:
- 10% → 25% → 50% → 100% traffic progression
- Feature flags for instant rollback
- Risk mitigation through staged releases

---
### 🔴 The Problem Gap: Why M8.3 Matters

**Real production failure scenario:**

Three independent changes deployed without comprehensive testing created a **compounding failure**:
1. Code refactoring introduced a chunking boundary bug
2. Embedding model upgrade changed semantic space
3. Prompt simplification worked with old embeddings but failed with new ones

**Result**: Quality dropped from **4.2/5 to 2.9/5** across 10,000 answers before detection.

**Core issue**: A/B testing validates changes individually, but production sees 5-10 commits daily from multiple engineers. **Manual testing cannot scale.**

M8.3 introduces **automated CI/CD pipelines** that catch regressions before they reach production.

---
## Section 2: Readiness Check #1 — Golden Test Set with Historical Baseline

**Purpose**: Establish a regression detection foundation with ground truth test cases.

**Requirements**:
- ✓ 100+ test cases with ground truth answers
- ✓ Baseline RAGAS metrics established (Faithfulness, Answer Relevance, Context Precision, Context Recall)
- ✓ Cases covering common queries, edge cases, and historical failures

**Impact if missing**: Cannot detect regressions automatically

### Check for Golden Test Set

Searches common file paths for golden test sets (JSON format). If found, reports test count. Otherwise, prints expected format and skips gracefully.

In [ ]:
import os
import json
from pathlib import Path

# Check for golden test set
test_set_paths = ['golden_test_set.json', 'test_data/golden_set.json', 'data/golden_tests.json']
golden_set_found = False

for path in test_set_paths:
    if Path(path).exists():
        print(f"✓ Found golden test set: {path}")
        with open(path) as f:
            data = json.load(f)
            count = len(data) if isinstance(data, list) else len(data.get('tests', []))
            print(f"  Test cases: {count}")
            golden_set_found = True
            break

if not golden_set_found:
    print("⚠️ Skipping (no golden test set found)")
    print("# Expected: golden_test_set.json with 100+ test cases")
    print("# Each case: {query, ground_truth, context, expected_metrics}")

# Expected: 
# ✓ Found golden test set: golden_test_set.json
#   Test cases: 125
#   Baseline metrics: Faithfulness=0.89, Answer Relevance=0.91

---
## Section 3: Readiness Check #2 — Version Control with Git

**Purpose**: Enable automated quality checks through version control integration.

**Requirements**:
- ✓ Code in GitHub/GitLab repository
- ✓ Pull request workflow (no direct main branch commits)
- ✓ Branch protection rules enabled
- ✓ Git command proficiency

**Impact if missing**: Cannot automate quality checks without version control

### Check Git Repository Status

Verifies current directory is a Git repository, checks for remote configuration, and notes that branch protection validation requires GitHub API access (offline-friendly skip).

In [ ]:
import subprocess
import os

# Check Git repository status
try:
    result = subprocess.run(['git', 'rev-parse', '--is-inside-work-tree'], 
                          capture_output=True, text=True, cwd=os.getcwd())
    if result.returncode == 0:
        print("✓ Git repository detected")
        
        # Check remote
        remote = subprocess.run(['git', 'remote', '-v'], capture_output=True, text=True)
        if remote.stdout:
            print(f"✓ Remote configured: {remote.stdout.splitlines()[0].split()[1]}")
        
        # Check branch protection (stub - requires GitHub API)
        print("⚠️ Branch protection check requires GitHub API token (skipping offline)")
    else:
        print("⚠️ Not a Git repository")
except FileNotFoundError:
    print("⚠️ Skipping (Git not installed)")

# Expected:
# ✓ Git repository detected
# ✓ Remote configured: git@github.com:org/rag-system.git
# ⚠️ Branch protection check requires GitHub API token (skipping offline)

---
## Section 4: Readiness Check #3 — Containerized RAG System

**Purpose**: Enable CI/CD test execution in isolated, reproducible environments.

**Requirements**:
- ✓ Working Dockerfile
- ✓ Successful Docker image builds
- ✓ Tests executable inside container
- ✓ All dependencies specified

**Impact if missing**: CI/CD cannot execute tests without containerization

### Check for Dockerfile and Docker Availability

Searches common Dockerfile locations. If found, verifies Docker CLI is installed. Does not attempt to build images (offline-friendly).

In [ ]:
# Check for Dockerfile
dockerfile_paths = ['Dockerfile', 'docker/Dockerfile', '.docker/Dockerfile']
dockerfile_found = False

for path in dockerfile_paths:
    if Path(path).exists():
        print(f"✓ Found Dockerfile: {path}")
        dockerfile_found = True
        break

if not dockerfile_found:
    print("⚠️ Skipping (no Dockerfile found)")
    print("# Expected: Dockerfile with test execution stage")
    
# Check Docker availability (offline-friendly - version check only)
try:
    docker_check = subprocess.run(['docker', '--version'], 
                                 capture_output=True, text=True, timeout=5)
    if docker_check.returncode == 0:
        print(f"✓ Docker available: {docker_check.stdout.strip()}")
    else:
        print("⚠️ Docker not available")
except (FileNotFoundError, subprocess.TimeoutExpired):
    print("⚠️ Skipping (Docker not installed)")

# Expected:
# ✓ Found Dockerfile: Dockerfile
# ✓ Docker available: Docker version 24.0.5

---
## Section 5: Readiness Check #4 — Fast Test Execution (<5 minutes)

**Purpose**: Prevent CI/CD bottlenecks with rapid feedback loops.

**Requirements**:
- ✓ Golden test set runs in <5 minutes
- ✓ Mock expensive external calls (OpenAI, Pinecone)
- ✓ Tests don't require full database setup
- ✓ Local test execution before pushing

**Impact if missing**: Slow CI becomes a workaround bottleneck

### Check for Test Files and Mocking Libraries

Detects test directories/files and verifies mocking capability. Does not execute tests (offline-friendly). Suggests running `pytest --durations=10` to measure test speed.

In [ ]:
import time

# Simulate test execution timing check (offline-friendly)
test_files = ['tests/', 'test/', 'tests.py', 'test_*.py']
test_found = any(Path(p).exists() for p in test_files)

if test_found:
    print("✓ Test files detected")
    print("⚠️ Run 'pytest --durations=10' to check test execution time")
else:
    print("⚠️ Skipping (no test files found)")
    print("# Expected: Test suite completes in <5 minutes")

# Check for mocking libraries
try:
    import unittest.mock
    print("✓ Mocking library available (unittest.mock)")
except ImportError:
    print("⚠️ Consider adding pytest-mock or unittest.mock")

# Expected:
# ✓ Test files detected
# ⚠️ Run 'pytest --durations=10' to check test execution time
# ✓ Mocking library available (unittest.mock)

---
## Section 6: Call-Forward — What M8.3 Will Introduce

M8.3 transforms manual testing into **automated regression detection** with four key capabilities:

### 🤖 Capability 1: GitHub Actions Workflow
**Automated pipeline triggering on every pull request**:
- Builds containers automatically
- Runs golden test sets
- Calculates RAGAS metrics
- Compares against baseline
- **Blocks PRs if quality degrades >5%**

**Outcome**: No manual intervention needed. Every PR validated before merge.

---

### 🛡️ Capability 2: Regression Detection with Baselines
**Automatic comparison of current metrics against baseline**:
- Blocks PRs with >5 percentage point Faithfulness drops
- Blocks PRs with doubled error rates
- Real-time quality gates in CI/CD pipeline
- Historical trend tracking

**Outcome**: Regressions caught in CI, never in production.

---

### 📦 Capability 3: Model & Prompt Versioning with DVC
**Version control for ML artifacts**:
- Embedding models versioned
- LLM models versioned
- Prompt templates tracked
- Golden test sets under version control
- **Instant rollback to last-known-good configuration in 2-3 minutes**

**Outcome**: Any change is reversible. Zero production risk.

---

### 💰 Capability 4: Cost & Latency Regression Detection
**Guardian rails for performance and economics**:
- Detects cost increases >30%
- Detects latency increases >25%
- Automated warnings or blocks
- Budget protection for production systems

**Outcome**: No surprise bills. No performance degradation.

---

### 🎯 The Ultimate Outcome

**"Zero regressions reach production. Every commit validated automatically. Engineers get feedback in 5 minutes."**

M8.3 shifts the burden from manual testing to automated pipelines, enabling teams to ship faster with higher confidence.

---
## Summary: Readiness Assessment

Run all cells above to validate your readiness for M8.3.

**Passing Criteria**:
- ✓ At least 2 of 4 checkpoints have infrastructure in place
- ✓ Understanding of why automation matters (problem gap)
- ✓ Clear vision of M8.3 capabilities (call-forward)

**Next Steps**:
1. Address any ⚠️ warnings from the checks above
2. Proceed to **M8.3: From Experimentation to Automation**
3. Implement automated CI/CD pipelines with regression detection

**Bridge Complete** ✅